In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('eventlog_large.csv', sep=';')
df.head()

,Case_ID,Activity,User,Timestamp
0,1,a,u2,2016-04-15 08:41:28
1,1,b,u3,2016-04-18 12:55:01
2,2,a,u2,2016-04-18 20:40:14
3,1,d,u5,2016-04-19 07:22:59
4,2,b,u4,2016-04-21 22:42:39


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45379 entries, 0 to 45378
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Case_ID    45379 non-null  int64 
 1   Activity   45379 non-null  object
 2   User       45379 non-null  object
 3   Timestamp  45379 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [4]:
#Convert the timestamp column to pandas datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45379 entries, 0 to 45378
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Case_ID    45379 non-null  int64         
 1   Activity   45379 non-null  object        
 2   User       45379 non-null  object        
 3   Timestamp  45379 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.4+ MB


In [5]:
df.head()

,Case_ID,Activity,User,Timestamp
0,1,a,u2,2016-04-15 08:41:28
1,1,b,u3,2016-04-18 12:55:01
2,2,a,u2,2016-04-18 20:40:14
3,1,d,u5,2016-04-19 07:22:59
4,2,b,u4,2016-04-21 22:42:39


In [6]:
#Create columns to extract the year, month and day from the timestamp column for further analysis
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day

df.head()

,Case_ID,Activity,User,Timestamp,Year,Month,Day
0,1,a,u2,2016-04-15 08:41:28,2016,4,15
1,1,b,u3,2016-04-18 12:55:01,2016,4,18
2,2,a,u2,2016-04-18 20:40:14,2016,4,18
3,1,d,u5,2016-04-19 07:22:59,2016,4,19
4,2,b,u4,2016-04-21 22:42:39,2016,4,21


In [7]:
#Sort the values by Case ID and arrange them according to the date
df_sort = df.copy()
df_sort = df_sort.sort_values(by=['Case_ID','Timestamp','Year','Month','Day','Activity'], ascending=True)
df_sort.head()

,Case_ID,Activity,User,Timestamp,Year,Month,Day
0,1,a,u2,2016-04-15 08:41:28,2016,4,15
1,1,b,u3,2016-04-18 12:55:01,2016,4,18
3,1,d,u5,2016-04-19 07:22:59,2016,4,19
5,1,e,u7,2016-04-23 15:06:58,2016,4,23
7,1,f,u7,2016-04-24 19:18:32,2016,4,24


In [8]:
#Group the dataframe by Case ID and create a count column that counts cummulatively how many occurences a case id has and assigns a number to each distinct activity of a case.
#This shows the number of activities each case id has
df_sort['Count'] = df_sort.groupby('Case_ID').cumcount()+1
df_sort

,Case_ID,Activity,User,Timestamp,Year,Month,Day,Count
0,1,a,u2,2016-04-15 08:41:28,2016,4,15,1
1,1,b,u3,2016-04-18 12:55:01,2016,4,18,2
3,1,d,u5,2016-04-19 07:22:59,2016,4,19,3
5,1,e,u7,2016-04-23 15:06:58,2016,4,23,4
7,1,f,u7,2016-04-24 19:18:32,2016,4,24,5
...,...,...,...,...,...,...,...,...
45371,7549,d,u5,2120-07-15 03:40:37,2120,7,15,3
45373,7549,g,u6,2120-07-19 05:39:32,2120,7,19,4
45374,7549,e,u7,2120-07-20 18:29:17,2120,7,20,5
45376,7549,f,u7,2120-07-22 03:43:02,2120,7,22,6


In [9]:
df_sort['Count'].max()

7

In [10]:
df_pivot = df_sort.copy()
df_pivot = df_pivot.pivot(index='Case_ID', columns='Count', values='Activity')
df_pivot

Count,1,2,3,4,5,6,7
Case_ID,,,,,,,
1,a,b,d,e,f,g,h
2,a,b,d,g,e,f,h
3,a,b,d,g,e,f,h
4,a,b,d,g,e,f,h
5,a,b,c,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7545,a,b,d,e,f,g,h
7546,a,b,d,e,f,g,h
7547,a,b,d,g,e,f,h


In [11]:
df_columns = df_pivot.columns
df_columns

Int64Index([1, 2, 3, 4, 5, 6, 7], dtype='int64', name='Count')

In [12]:
df_pivot = df_pivot.fillna('X')
df_pivot = df_pivot.astype('str')
df_pivot['Trace'] = df_pivot.apply(lambda x: ','.join(x), axis=1)
df_pivot['Trace'] = df_pivot['Trace'].apply(lambda x: x.replace(',X',''))

df_pivot

Count,1,2,3,4,5,6,7,Trace
Case_ID,,,,,,,,
1,a,b,d,e,f,g,h,"a,b,d,e,f,g,h"
2,a,b,d,g,e,f,h,"a,b,d,g,e,f,h"
3,a,b,d,g,e,f,h,"a,b,d,g,e,f,h"
4,a,b,d,g,e,f,h,"a,b,d,g,e,f,h"
5,a,b,c,X,X,X,X,"a,b,c"
...,...,...,...,...,...,...,...,...
7545,a,b,d,e,f,g,h,"a,b,d,e,f,g,h"
7546,a,b,d,e,f,g,h,"a,b,d,e,f,g,h"
7547,a,b,d,g,e,f,h,"a,b,d,g,e,f,h"


In [13]:
df_pivot_2=df_pivot[['Trace',1]].groupby(['Trace'], as_index=False).count()
df_pivot_2

Count,Trace,1
0,"a,b,c",1866
1,"a,b,d,e,f,g,h",1861
2,"a,b,d,e,g,f,h",959
3,"a,b,d,g,e,f,h",2863


In [14]:
df_pivot_2 = df_pivot_2.sort_values(by=1, ascending=False).reset_index()
df_pivot_2

Count,index,Trace,1
0,3,"a,b,d,g,e,f,h",2863
1,0,"a,b,c",1866
2,1,"a,b,d,e,f,g,h",1861
3,2,"a,b,d,e,g,f,h",959


In [15]:
del df_pivot_2['index']
df_pivot_2 = df_pivot_2.reset_index()
df_pivot_2['index'] = df_pivot_2['index']+1
df_pivot_2

Count,index,Trace,1
0,1,"a,b,d,g,e,f,h",2863
1,2,"a,b,c",1866
2,3,"a,b,d,e,f,g,h",1861
3,4,"a,b,d,e,g,f,h",959


In [16]:
df_pivot_2 = df_pivot_2.rename(columns={'index':'Variants',1:'Count'})
df_pivot_2

Count,Variants,Trace,Count
0,1,"a,b,d,g,e,f,h",2863
1,2,"a,b,c",1866
2,3,"a,b,d,e,f,g,h",1861
3,4,"a,b,d,e,g,f,h",959


In [17]:
df_pivot_2.to_csv('Case_Count.csv', index=False)

In [18]:
#Df_case
df_sort

,Case_ID,Activity,User,Timestamp,Year,Month,Day,Count
0,1,a,u2,2016-04-15 08:41:28,2016,4,15,1
1,1,b,u3,2016-04-18 12:55:01,2016,4,18,2
3,1,d,u5,2016-04-19 07:22:59,2016,4,19,3
5,1,e,u7,2016-04-23 15:06:58,2016,4,23,4
7,1,f,u7,2016-04-24 19:18:32,2016,4,24,5
...,...,...,...,...,...,...,...,...
45371,7549,d,u5,2120-07-15 03:40:37,2120,7,15,3
45373,7549,g,u6,2120-07-19 05:39:32,2120,7,19,4
45374,7549,e,u7,2120-07-20 18:29:17,2120,7,20,5
45376,7549,f,u7,2120-07-22 03:43:02,2120,7,22,6


In [19]:
df_case = df_sort.copy()

In [20]:
df_case = df_case.pivot(index='Case_ID',columns='Count', values='Timestamp')
df_case

Count,1,2,3,4,5,6,7
Case_ID,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...
7545,2120-06-24 17:36:03,2120-06-25 11:13:57,2120-06-26 00:22:29,2120-06-28 04:06:45,2120-06-28 22:15:25,2120-06-28 22:55:12,2120-07-02 01:29:52
7546,2120-07-01 11:45:11,2120-07-02 00:35:36,2120-07-03 22:47:22,2120-07-04 21:58:41,2120-07-05 03:37:20,2120-07-09 10:36:39,2120-07-11 13:00:03
7547,2120-07-04 17:07:06,2120-07-06 16:22:39,2120-07-10 04:24:14,2120-07-13 20:46:17,2120-07-16 06:18:37,2120-07-21 23:42:12,2120-07-22 22:17:09


In [21]:
df_case['Max'] = df_case.apply(max, axis=1)
df_case

Count,1,2,3,4,5,6,7,Max
Case_ID,,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36,2016-04-26 10:00:36
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18,2016-05-01 18:32:18
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53,2016-05-02 11:06:53
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28,2016-05-17 14:02:28
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT,2016-05-11 05:04:34
...,...,...,...,...,...,...,...,...
7545,2120-06-24 17:36:03,2120-06-25 11:13:57,2120-06-26 00:22:29,2120-06-28 04:06:45,2120-06-28 22:15:25,2120-06-28 22:55:12,2120-07-02 01:29:52,2120-07-02 01:29:52
7546,2120-07-01 11:45:11,2120-07-02 00:35:36,2120-07-03 22:47:22,2120-07-04 21:58:41,2120-07-05 03:37:20,2120-07-09 10:36:39,2120-07-11 13:00:03,2120-07-11 13:00:03
7547,2120-07-04 17:07:06,2120-07-06 16:22:39,2120-07-10 04:24:14,2120-07-13 20:46:17,2120-07-16 06:18:37,2120-07-21 23:42:12,2120-07-22 22:17:09,2120-07-22 22:17:09


In [22]:
df_case['Case Duration'] = df_case['Max'] - df_case[1]
df_case

Count,1,2,3,4,5,6,7,Max,Case Duration
Case_ID,,,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36,2016-04-26 10:00:36,11 days 01:19:08
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18,2016-05-01 18:32:18,12 days 21:52:04
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53,2016-05-02 11:06:53,5 days 09:21:46
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28,2016-05-17 14:02:28,15 days 05:23:54
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT,2016-05-11 05:04:34,1 days 21:47:22
...,...,...,...,...,...,...,...,...,...
7545,2120-06-24 17:36:03,2120-06-25 11:13:57,2120-06-26 00:22:29,2120-06-28 04:06:45,2120-06-28 22:15:25,2120-06-28 22:55:12,2120-07-02 01:29:52,2120-07-02 01:29:52,7 days 07:53:49
7546,2120-07-01 11:45:11,2120-07-02 00:35:36,2120-07-03 22:47:22,2120-07-04 21:58:41,2120-07-05 03:37:20,2120-07-09 10:36:39,2120-07-11 13:00:03,2120-07-11 13:00:03,10 days 01:14:52
7547,2120-07-04 17:07:06,2120-07-06 16:22:39,2120-07-10 04:24:14,2120-07-13 20:46:17,2120-07-16 06:18:37,2120-07-21 23:42:12,2120-07-22 22:17:09,2120-07-22 22:17:09,18 days 05:10:03


In [23]:
# df_case['Case Duration'] = df_case['Case Duration'].apply(lambda x: round(x.total_seconds(),0))
df_case

Count,1,2,3,4,5,6,7,Max,Case Duration
Case_ID,,,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36,2016-04-26 10:00:36,11 days 01:19:08
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18,2016-05-01 18:32:18,12 days 21:52:04
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53,2016-05-02 11:06:53,5 days 09:21:46
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28,2016-05-17 14:02:28,15 days 05:23:54
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT,2016-05-11 05:04:34,1 days 21:47:22
...,...,...,...,...,...,...,...,...,...
7545,2120-06-24 17:36:03,2120-06-25 11:13:57,2120-06-26 00:22:29,2120-06-28 04:06:45,2120-06-28 22:15:25,2120-06-28 22:55:12,2120-07-02 01:29:52,2120-07-02 01:29:52,7 days 07:53:49
7546,2120-07-01 11:45:11,2120-07-02 00:35:36,2120-07-03 22:47:22,2120-07-04 21:58:41,2120-07-05 03:37:20,2120-07-09 10:36:39,2120-07-11 13:00:03,2120-07-11 13:00:03,10 days 01:14:52
7547,2120-07-04 17:07:06,2120-07-06 16:22:39,2120-07-10 04:24:14,2120-07-13 20:46:17,2120-07-16 06:18:37,2120-07-21 23:42:12,2120-07-22 22:17:09,2120-07-22 22:17:09,18 days 05:10:03


In [24]:
df_columns

Int64Index([1, 2, 3, 4, 5, 6, 7], dtype='int64', name='Count')

In [25]:
max_length = df_sort['Count'].max()

In [26]:
df_case['Case Length'] = max_length - df_case.isnull().sum(axis=1)
df_case

Count,1,2,3,4,5,6,7,Max,Case Duration,Case Length
Case_ID,,,,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36,2016-04-26 10:00:36,11 days 01:19:08,7
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18,2016-05-01 18:32:18,12 days 21:52:04,7
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53,2016-05-02 11:06:53,5 days 09:21:46,7
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28,2016-05-17 14:02:28,15 days 05:23:54,7
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT,2016-05-11 05:04:34,1 days 21:47:22,3
...,...,...,...,...,...,...,...,...,...,...
7545,2120-06-24 17:36:03,2120-06-25 11:13:57,2120-06-26 00:22:29,2120-06-28 04:06:45,2120-06-28 22:15:25,2120-06-28 22:55:12,2120-07-02 01:29:52,2120-07-02 01:29:52,7 days 07:53:49,7
7546,2120-07-01 11:45:11,2120-07-02 00:35:36,2120-07-03 22:47:22,2120-07-04 21:58:41,2120-07-05 03:37:20,2120-07-09 10:36:39,2120-07-11 13:00:03,2120-07-11 13:00:03,10 days 01:14:52,7
7547,2120-07-04 17:07:06,2120-07-06 16:22:39,2120-07-10 04:24:14,2120-07-13 20:46:17,2120-07-16 06:18:37,2120-07-21 23:42:12,2120-07-22 22:17:09,2120-07-22 22:17:09,18 days 05:10:03,7


In [27]:
# 
df_case = df_case.drop(df_columns, axis=1)
df_case

Count,Max,Case Duration,Case Length
Case_ID,,,
1,2016-04-26 10:00:36,11 days 01:19:08,7
2,2016-05-01 18:32:18,12 days 21:52:04,7
3,2016-05-02 11:06:53,5 days 09:21:46,7
4,2016-05-17 14:02:28,15 days 05:23:54,7
5,2016-05-11 05:04:34,1 days 21:47:22,3
...,...,...,...
7545,2120-07-02 01:29:52,7 days 07:53:49,7
7546,2120-07-11 13:00:03,10 days 01:14:52,7
7547,2120-07-22 22:17:09,18 days 05:10:03,7


In [28]:
df_pivot_3 = df_sort.copy()
df_pivot_3 = df_pivot_3.pivot(index='Case_ID', columns='Count', values='Activity')
df_pivot_3

Count,1,2,3,4,5,6,7
Case_ID,,,,,,,
1,a,b,d,e,f,g,h
2,a,b,d,g,e,f,h
3,a,b,d,g,e,f,h
4,a,b,d,g,e,f,h
5,a,b,c,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7545,a,b,d,e,f,g,h
7546,a,b,d,e,f,g,h
7547,a,b,d,g,e,f,h


In [29]:
def last_activity(a):
    if a.last_valid_index() is None:
        return np.nan
    else:
        return a[a.last_valid_index()]

In [30]:
df_pivot_3['Last Activity'] = df_pivot_3.apply(last_activity, axis=1)
df_pivot_3['First Activity'] = df_pivot_3[1]
df_pivot_3

Count,1,2,3,4,5,6,7,Last Activity,First Activity
Case_ID,,,,,,,,,
1,a,b,d,e,f,g,h,h,a
2,a,b,d,g,e,f,h,h,a
3,a,b,d,g,e,f,h,h,a
4,a,b,d,g,e,f,h,h,a
5,a,b,c,NaN,NaN,NaN,NaN,c,a
...,...,...,...,...,...,...,...,...,...
7545,a,b,d,e,f,g,h,h,a
7546,a,b,d,e,f,g,h,h,a
7547,a,b,d,g,e,f,h,h,a


In [31]:
df_case = df_case.drop(['Max'],axis=1)

In [32]:
df_pivot_3 = df_pivot_3.drop(df_columns,axis=1)

In [33]:
df_pivot_4 = df_pivot.copy()
df_pivot_4 = df_pivot_4.reset_index()
df_pivot_4 = df_pivot_4[['Case_ID','Trace']]
df_pivot_4

Count,Case_ID,Trace
0,1,"a,b,d,e,f,g,h"
1,2,"a,b,d,g,e,f,h"
2,3,"a,b,d,g,e,f,h"
3,4,"a,b,d,g,e,f,h"
4,5,"a,b,c"
...,...,...
7544,7545,"a,b,d,e,f,g,h"
7545,7546,"a,b,d,e,f,g,h"
7546,7547,"a,b,d,g,e,f,h"
7547,7548,"a,b,c"


In [34]:
df_pivot_5 = df_pivot_3.merge(df_pivot_4, left_on='Case_ID', right_on='Case_ID', how='inner')

In [35]:
df_case_2 = df_pivot_5.merge(df_case, left_on='Case_ID', right_on='Case_ID', how='inner')

In [36]:
df_pivot_2

Count,Variants,Trace,Count
0,1,"a,b,d,g,e,f,h",2863
1,2,"a,b,c",1866
2,3,"a,b,d,e,f,g,h",1861
3,4,"a,b,d,e,g,f,h",959


In [37]:
df_case_2.head()

Count,Case_ID,Last Activity,First Activity,Trace,Case Duration,Case Length
0,1,h,a,"a,b,d,e,f,g,h",11 days 01:19:08,7
1,2,h,a,"a,b,d,g,e,f,h",12 days 21:52:04,7
2,3,h,a,"a,b,d,g,e,f,h",5 days 09:21:46,7
3,4,h,a,"a,b,d,g,e,f,h",15 days 05:23:54,7
4,5,c,a,"a,b,c",1 days 21:47:22,3


In [38]:
df_case_3 = df_case_2.merge(df_pivot_2, left_on='Trace', right_on='Trace', how='outer').sort_values(by='Case_ID')

In [39]:
df_case_3 = df_case_3[['Case_ID','Case Duration','Case Length','Variants','First Activity','Last Activity']]
df_case_3['Case Duration'] = df_case_3['Case Duration'].apply(lambda x: round(x.total_seconds(),0))
df_case_3

Count,Case_ID,Case Duration,Case Length,Variants,First Activity,Last Activity
0,1,955148.0,7,3,a,h
1861,2,1115524.0,7,1,a,h
1862,3,465706.0,7,1,a,h
1863,4,1315434.0,7,1,a,h
4724,5,164842.0,3,2,a,c
...,...,...,...,...,...,...
1859,7545,633229.0,7,3,a,h
1860,7546,868492.0,7,3,a,h
4722,7547,1573803.0,7,1,a,h
6589,7548,141827.0,3,2,a,c


In [40]:
df_case_3.to_csv('Case.csv', index=False)

In [41]:
#Df node
df_node = df.copy()
df_node = df_node[['Case_ID','Activity','Timestamp','User']]
df_node

,Case_ID,Activity,Timestamp,User
0,1,a,2016-04-15 08:41:28,u2
1,1,b,2016-04-18 12:55:01,u3
2,2,a,2016-04-18 20:40:14,u2
3,1,d,2016-04-19 07:22:59,u5
4,2,b,2016-04-21 22:42:39,u4
...,...,...,...,...
45374,7549,e,2120-07-20 18:29:17,u7
45375,7547,f,2120-07-21 23:42:12,u7
45376,7549,f,2120-07-22 03:43:02,u7
45377,7547,h,2120-07-22 22:17:09,u1


In [42]:
df_node = df_node.sort_values(by=['Case_ID','Timestamp','Activity'],ascending=True).reset_index(drop=True)
df_node

,Case_ID,Activity,Timestamp,User
0,1,a,2016-04-15 08:41:28,u2
1,1,b,2016-04-18 12:55:01,u3
2,1,d,2016-04-19 07:22:59,u5
3,1,e,2016-04-23 15:06:58,u7
4,1,f,2016-04-24 19:18:32,u7
...,...,...,...,...
45374,7549,d,2120-07-15 03:40:37,u5
45375,7549,g,2120-07-19 05:39:32,u6
45376,7549,e,2120-07-20 18:29:17,u7
45377,7549,f,2120-07-22 03:43:02,u7


In [43]:
sort1 = df_node.iloc[:-1,:]
sort1.head()

,Case_ID,Activity,Timestamp,User
0,1,a,2016-04-15 08:41:28,u2
1,1,b,2016-04-18 12:55:01,u3
2,1,d,2016-04-19 07:22:59,u5
3,1,e,2016-04-23 15:06:58,u7
4,1,f,2016-04-24 19:18:32,u7


In [44]:
sort2 = df_node.iloc[1:,:].reset_index(drop=True)
sort2.head()

,Case_ID,Activity,Timestamp,User
0,1,b,2016-04-18 12:55:01,u3
1,1,d,2016-04-19 07:22:59,u5
2,1,e,2016-04-23 15:06:58,u7
3,1,f,2016-04-24 19:18:32,u7
4,1,g,2016-04-25 14:56:42,u5


In [45]:
sort = pd.concat([sort1,sort2],axis=1)
sort

,Case_ID,Activity,Timestamp,User,Case_ID,Activity,Timestamp,User
0,1,a,2016-04-15 08:41:28,u2,1,b,2016-04-18 12:55:01,u3
1,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5
2,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7
3,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7
4,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5
...,...,...,...,...,...,...,...,...
45373,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5
45374,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6
45375,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7
45376,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7


In [46]:
sort.columns = ['Case_ID','Source','Timestamp1','User1','Case_ID2','Target','Timestamp2','User2']
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2
0,1,a,2016-04-15 08:41:28,u2,1,b,2016-04-18 12:55:01,u3
1,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5
2,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7
3,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7
4,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5
...,...,...,...,...,...,...,...,...
45373,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5
45374,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6
45375,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7
45376,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7


In [47]:
sort['Comparison'] = sort.apply(lambda x: True if x['Case_ID'] == x['Case_ID2'] else False,axis=1)
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison
0,1,a,2016-04-15 08:41:28,u2,1,b,2016-04-18 12:55:01,u3,True
1,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True
2,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True
3,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True
4,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True
...,...,...,...,...,...,...,...,...,...
45373,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True
45374,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True
45375,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True
45376,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True


In [48]:
sort = sort[sort['Comparison'] == True]

In [49]:
sort['Count'] = sort.groupby('Case_ID').cumcount() + 1
sort

<ipython-input-49-4aa66f3a05a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sort['Count'] = sort.groupby('Case_ID').cumcount() + 1


,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count
0,1,a,2016-04-15 08:41:28,u2,1,b,2016-04-18 12:55:01,u3,True,1
1,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2
2,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3
3,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4
4,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5
...,...,...,...,...,...,...,...,...,...,...
45373,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2
45374,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3
45375,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4
45376,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5


In [50]:
sort = sort.sort_values(by=['Case_ID','Count'], ascending=[True,False])
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count
5,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6
4,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5
3,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4
2,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3
1,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2
...,...,...,...,...,...,...,...,...,...,...
45376,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5
45375,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4
45374,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3
45373,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2


In [51]:
sort.reset_index(drop=True)

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count
0,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6
1,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5
2,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4
3,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3
4,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2
...,...,...,...,...,...,...,...,...,...,...
37825,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5
37826,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4
37827,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3
37828,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2


In [52]:
sort['Count_Reverse'] = sort.groupby('Case_ID').cumcount()+1
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count,Count_Reverse
5,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6,1
4,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5,2
3,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4,3
2,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3,4
1,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2,5
...,...,...,...,...,...,...,...,...,...,...,...
45376,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5,2
45375,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4,3
45374,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3,4
45373,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2,5


In [53]:
sort = sort.sort_values(by=['Case_ID', 'Count'], ascending=[True,False]).reset_index(drop=True)
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count,Count_Reverse
0,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6,1
1,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5,2
2,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4,3
3,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3,4
4,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2,5
...,...,...,...,...,...,...,...,...,...,...,...
37825,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5,2
37826,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4,3
37827,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3,4
37828,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2,5


In [54]:
sort['Start_Connect'] = sort['Count'].apply(lambda x: 'Start' if x==1 else '')
sort['End_Connect'] = sort['Count_Reverse'].apply(lambda x: 'End' if x==1 else '')
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count,Count_Reverse,Start_Connect,End_Connect
0,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6,1,,End
1,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5,2,,
2,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4,3,,
3,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3,4,,
4,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2,5,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37825,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5,2,,
37826,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4,3,,
37827,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3,4,,
37828,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2,5,,


In [55]:
sort['Duration'] = sort['Timestamp1'] - sort['Timestamp2']
sort['Duration'] = sort['Duration'].apply(lambda x: round(x.total_seconds(),0))
sort

,Case_ID,Source,Timestamp1,User1,Case_ID2,Target,Timestamp2,User2,Comparison,Count,Count_Reverse,Start_Connect,End_Connect,Duration
0,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6,1,,End,-68634.0
1,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5,2,,,-70690.0
2,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4,3,,,-101494.0
3,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3,4,,,-373439.0
4,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2,5,,,-66478.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37825,7549,e,2120-07-20 18:29:17,u7,7549,f,2120-07-22 03:43:02,u7,True,5,2,,,-119625.0
37826,7549,g,2120-07-19 05:39:32,u6,7549,e,2120-07-20 18:29:17,u7,True,4,3,,,-132585.0
37827,7549,d,2120-07-15 03:40:37,u5,7549,g,2120-07-19 05:39:32,u6,True,3,4,,,-352735.0
37828,7549,b,2120-07-14 22:42:16,u3,7549,d,2120-07-15 03:40:37,u5,True,2,5,,,-17901.0


In [56]:
sort.to_csv('Node.csv', index=False)